In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "proposals/file_1.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

16


In [22]:
import os

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False
    )
    return text_splitter.split_documents(documents)    


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
import os
import chromadb

# Initialize ChromaDB client
client = chromadb.Client()

# Create a collection in ChromaDB
collection = client.create_collection("pdf_embeddings")

# Get the list of PDF files in the proposals folder
proposals_folder = "proposals"
pdf_files = [f for f in os.listdir(proposals_folder) if f.endswith('.pdf')]

# Initialize the embeddings model
embeddings_model = OpenAIEmbeddings(api_key=os.getenv("OPEN_API_KEY"))

# Process each PDF file
for pdf_file in pdf_files[:2]:
    try:
        file_path = os.path.join(proposals_folder, pdf_file)
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        
        # Embed the content of each document
        for doc in docs:
            embedding = embeddings_model.embed_documents([doc.page_content])[0]
            
            # Store the embedding in ChromaDB
            collection.add(doc.metadata['source'], embedding)
    except Exception as e:
        print(f"Error processing {pdf_file}: {e}")

print("All PDF files have been embedded and stored in ChromaDB.")

Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
Add of existing embedding ID: proposals/file_6.pdf
Insert of existing embedding ID: proposals/file_6.pdf
A

All PDF files have been embedded and stored in ChromaDB.


In [36]:
embedding[0]

-0.033174023032188416

In [ ]:
query = "Why use Yolov8"
docs = db.similarity_search(query)
print(docs[0].page_content)


In [33]:
# Define the query
query = "Why use Yolov8"
def searchDataByVector(query: str):
    try:
        query_vector = embeddings_model.embed_query(query)
        res = collection.query(
            query_embeddings=[query_vector],
            n_results=1,
            include=['distances','embeddings', 'documents', 'metadatas'],
        )
        print("Query", "\n--------------")
        print(query)
        print("Result", "\n--------------")
        print(res['documents'][0][0])
        print("Vector", "\n--------------")
        print(res['embeddings'][0][0])
        print("")
        print("")
        print("Complete Response","\n-------------------------")
        print(res)

    except Exception as e:
        print("Vector search failed : ", e)

searchDataByVector(query)

Query 
--------------
Why use Yolov8
Result 
--------------
None
Vector 
--------------
[-0.03085641  0.00316999  0.00816987 ... -0.01453027 -0.00579238
 -0.00167938]


Complete Response 
-------------------------
{'ids': [['proposals/file_21.pdf']], 'embeddings': [array([[-0.03085641,  0.00316999,  0.00816987, ..., -0.01453027,
        -0.00579238, -0.00167938]])], 'documents': [[None]], 'uris': None, 'data': None, 'metadatas': [[None]], 'distances': [[0.5352519750595093]], 'included': [<IncludeEnum.embeddings: 'embeddings'>, <IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
